In [2]:
# Import data
#Importing relevant libraries for the analysis
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
from urllib.request import urlopen
import xlrd

In [9]:
#Importing and Concating CDC Data 
all_msa = []

for num in range(0,56):
    all_msa.append(pd.read_table('United States and Puerto Rico Cancer Statistics, 1999-2017 Incidence ({}).txt'.format(num)))
    
Extract = pd.concat(all_msa)

In [10]:
Extract.shape

(40877, 12)

In [184]:
Extract.head()

,Notes,Cancer Sites,Cancer Sites Code,MSA,MSA Code,Age Groups,Age Groups Code,Sex,Sex Code,Race,Race Code,Count
0,NaN,All Invasive Cancer Sites Combined,0,"Akron, OH",10420.0,25-29 years,25-29,Female,F,White,2106-3,17.0
1,NaN,All Invasive Cancer Sites Combined,0,"Akron, OH",10420.0,30-34 years,30-34,Female,F,White,2106-3,26.0
2,NaN,All Invasive Cancer Sites Combined,0,"Akron, OH",10420.0,35-39 years,35-39,Female,F,White,2106-3,31.0
3,NaN,All Invasive Cancer Sites Combined,0,"Akron, OH",10420.0,35-39 years,35-39,Male,M,White,2106-3,22.0
4,NaN,All Invasive Cancer Sites Combined,0,"Akron, OH",10420.0,40-44 years,40-44,Female,F,White,2106-3,58.0


In [185]:
#Importing CDC data for cleaning and processing
CDC_DF = Extract.drop(['Notes'], axis = 1)

CDC_DF['Count'].replace('', np.nan, inplace=True)
CDC_DF.dropna(subset=['Count'], inplace=True)
CDC_DF = CDC_DF[CDC_DF['Cancer Sites'] != 'All Invasive Cancer Sites Combined']


CDC_DF = CDC_DF.rename(columns = {'Cancer Sites' : 'Cancer_Sites', 'Cancer Sites Code' : 'Cancer_Sites_Code','MSA Code': 'MSA_Code',
                                'Age Groups' : 'Age_Groups', 'Age Groups Code' : 'Age_Groups_Code', 'Sex Code' : 'Sex_Code',
                                 'Race Code' : 'Race_Code'})
CDC_DF.shape

(32849, 11)

In [186]:
#After looking through the CDC cancer sites these are the cancer_sites level of detail I chose to view 
Cancer_list = ('Oral Cavity and Pharynx','Esophagus','Stomach','Small Intestine','Colon and Rectum',
'Anus, Anal Canal, and Anorectum','Liver and Intrahepatic Bile Duct','Gallbladder','Other Biliary', 'Pancreas',
'Retroperitoneum','Peritoneum, Omentum and Mesentery','Other Digestive Organs','Nose, Nasal Cavity and Middle Ear','Larynx',
'Lung and Bronchus','Pleura','Trachea, Mediastinum and Other Respiratory Organs','Bones and Joints',
'Soft Tissue including Heart','Skin excluding Basal and Squamous','Male and Female Breast','Female Genital System',
'Male Genital System','Urinary System', 'Eye and Orbit', 'Brain and Other Nervous System', 'Endocrine System', 'Lymphomas',
'Myeloma', 'Leukemias', 'Mesothelioma', 'Kaposi Sarcoma', 'Miscellaneous', 'Male and Female Breast, In Situ')

In [187]:
#Making Age Groups to be every ten years
CDC_DF['Age_Groups'].replace(['55-59 years', '60-64 years ', '65-69 years', '70-74 years',
       '80-84 years', '85+ years', '50-54 years', '75-79 years',
       '40-44 years', '45-49 years', '35-39 years', '25-29 years',
       '30-34 years', '20-24 years', '15-19 years', '1-4 years',
       '5-9 years', '10-14 years', '< 1 year'],['50-59 years', '60-69 years', '60-69 years', '70-79 years',
       '80+', '80+', '50-59 years', '70-79 years',
       '40-49 years', '40-49 years', '30-39 years', '20-29 years',
       '30-39 years', '20-29 years', '10-19 years', '0-9 years',
       '0-9 years', '10-19 years', '0-9 years'] , inplace = True)

In [188]:
boolean_series = CDC_DF['Cancer_Sites'].isin(Cancer_list)

CDC_DF = CDC_DF[boolean_series]
CDC_DF.shape

(15475, 11)

In [229]:
#Importing in the seer data and doing  data cleaning
seer_df = pd.read_excel('5 year survival rate SEER data.xls')
seer_df['Cancer Sites'] = seer_df['Cancer Sites'].fillna(method='ffill')
seer_df['Race'] = seer_df['Race'].fillna(method='ffill')

seer_df = seer_df.rename(columns = {'Cancer Sites' : 'Cancer_Sites' , 
                                    '5-Year conditional relative survival' : '5-Year_Conditional_Relative_Survival'})

seer_df = seer_df.drop(['Number of cases surviving the conditional period'], axis = 1)
seer_df.head()

,Cancer_Sites,Race,Gender,5-Year_Conditional_Relative_Survival
0,All Sites,All races,Male and female,0.929
1,All Sites,All races,Male,0.938
2,All Sites,All races,Female,0.92
3,All Sites,White,Male and female,0.933
4,All Sites,White,Male,0.941


In [230]:
#For compare to see what column names need to be updated to have similar naming conventions between SEER and CDC
cdc_list = CDC_DF['Cancer_Sites'].unique().tolist()
seer_list = seer_df['Cancer_Sites'].unique().tolist()
seer_diff_list = seer_df['Cancer_Sites'].unique().tolist()

for item in cdc_list:
    if item in seer_list:
        seer_diff_list.remove(item)
seer_diff_list

['All Sites',
 'Melanoma of the Skin',
 'Breast',
 'Cervix Uteri',
 'Corpus and Uterus, NOS',
 'Ovary',
 'Prostate',
 'Testis',
 'Urinary Bladder',
 'Kidney and Renal Pelvis',
 'Thyroid',
 'Hodgkin Lymphoma',
 'Non-Hodgkin Lymphoma',
 'Leukemia',
 'Lymphocytic Leukemia',
 'Acute Lymphocytic Leukemia',
 'Chronic Lymphocytic Leukemia',
 'Myeloid and Monocytic Leukemia',
 'Acute Myeloid Leukemia',
 'Acute Monocytic Leukemia',
 'Chronic Myeloid Leukemia',
 'Kaposi Sarcoma']

In [231]:
#Adjust seer data titles to have similar conventions as CDC
seer_df['Cancer_Sites'].replace( ['Breast', 'Leukemia', 'Thyroid', 'Urinary Bladder',],
                                ['Male and Female Breast','Leukemias','Endocrine System', 'Urinary System'], inplace = True)

In [232]:
#Adjusting the list of cancer sites in the SEER to better align with CDC. 
cancer_list = CDC_DF['Cancer_Sites'].unique().tolist()
additions = 'Cervix Uteri','Corpus and Uterus, NOS','Ovary','Prostate','Testis'
cancer_list += additions

seer_df = seer_df[seer_df['Cancer_Sites'].isin(cancer_list)]

In [233]:
#bringing in data for MSA population
pop_df = pd.read_excel('MSA Population data.xlsx')
pop_df = pop_df.rename(columns={'CBSA': 'MSA_Code'})

pop_df = pop_df.groupby(['MSA_Code']).sum()

pop_df.head()

,Population
MSA_Code,
10100,85678
10140,143062
10180,340034
10220,77024
10300,196992


In [234]:
#Creating CDC based dataframes for background and for plotting how different demographics compare in case count
MSA_df0 = CDC_DF.groupby(['Cancer_Sites','MSA', 'MSA_Code']).sum().reset_index()
MSA_df1 = CDC_DF.groupby(['MSA', 'MSA_Code']).sum().reset_index()
MSA_Cancer_df = MSA_df0.merge(pop_df, how='left', on='MSA_Code')
MSA_df = MSA_df1.merge(pop_df, how='left', on='MSA_Code')

MSA_df['%_Population'] = MSA_df['Count'] / MSA_df['Population']

CDC_DF2 = CDC_DF.drop(['MSA_Code'], axis = 1)

Cancer_df = CDC_DF2.groupby(['Cancer_Sites']).sum().reset_index()

Age_Cancer_df = CDC_DF2.groupby(['Cancer_Sites', 'Age_Groups']).sum().reset_index()
Age_df = CDC_DF2.groupby(['Age_Groups']).sum().reset_index()

Gender_Cancer_df = CDC_DF2.groupby(['Cancer_Sites', 'Sex', 'Sex_Code']).sum().reset_index()
Gender_df = CDC_DF2.groupby(['Sex', 'Sex_Code']).sum().reset_index()

Race_Cancer_df = CDC_DF2.groupby(['Cancer_Sites', 'Race']).sum().reset_index()
Race_df = CDC_DF2.groupby(['Race']).sum().reset_index()

In [244]:
MSA_df.sort_values(by = '%_Population', ascending = False).head()

,MSA,MSA_Code,Count,Population,%_Population
64,"North Port-Sarasota-Bradenton, FL",35840.0,4756.0,1576682.0,0.003016
74,"Pittsburgh, PA",38300.0,13120.0,4680444.0,0.002803
98,"Tampa-St. Petersburg-Clearwater, FL",45300.0,16239.0,6097926.0,0.002663
14,"Buffalo-Cheektowaga-Niagara Falls, NY",15380.0,5763.0,2260464.0,0.002549
27,"Deltona-Daytona Beach-Ormond Beach, FL",19660.0,3214.0,1271718.0,0.002527


In [236]:
#Creating seer based dataframes for background and for plotting how different demographics compare in survival rates
seer_Cancer_df = seer_df.loc[(seer_df['Cancer_Sites'] != 'All Sites') &
                         (seer_df['Race'] == 'All races') &
                         (seer_df['Gender'] == 'Male and female')].reset_index()

seer_Race_Cancer_df = seer_df.loc[(seer_df['Cancer_Sites'] != 'All Sites') &
                         (seer_df['Race'] != 'All races') &
                         (seer_df['Gender'] == 'Male and female')].reset_index()
seer_Race_df = seer_df.loc[(seer_df['Cancer_Sites'] == 'All Sites') &
                         (seer_df['Race'] != 'All races') &
                         (seer_df['Gender'] == 'Male and female')].reset_index()

seer_Gender_Cancer_df = seer_df.loc[(seer_df['Cancer_Sites'] != 'All Sites') &
                         (seer_df['Race'] == 'All races') &
                         (seer_df['Gender'] != 'Male and female')].reset_index()
seer_Gender_df = seer_df.loc[(seer_df['Cancer_Sites'] == 'All Sites') &
                         (seer_df['Race'] == 'All races') &
                         (seer_df['Gender'] != 'Male and female')].reset_index()

seer_total_df = seer_df.loc[(seer_df['Cancer_Sites'] != 'All Sites') &
                         (seer_df['Race'] != 'All races') &
                         (seer_df['Gender'] != 'Male and female')].reset_index()

In [237]:
#creating an MSA dataframe for mapping MSA Data
MSA_complete_df = MSA_Cancer_df.merge( seer_Cancer_df, how = 'left',left_on = 'Cancer_Sites', right_on = 'Cancer_Sites')
MSA_complete_df = MSA_complete_df.drop(['Race','Gender','index'], axis = 1)
MSA_complete_df['Cases Per million People'] = MSA_complete_df['Count'] / (MSA_complete_df['Population']/1000000)
MSA_complete_df = MSA_complete_df.fillna(0)
MSA_complete_df.head()

,Cancer_Sites,MSA,MSA_Code,Count,Population,5-Year_Conditional_Relative_Survival,Cases Per million People
0,Bones and Joints,"New York-Newark-Jersey City, NY-NJ-PA",35620.0,17.0,58004334.0,0.912,0.293082
1,Bones and Joints,Other,99999.0,890.0,0.0,0.912,0.000000
2,Brain and Other Nervous System,"Boston-Cambridge-Newton, MA-NH",14460.0,156.0,14427183.0,0.842,10.812922
3,Brain and Other Nervous System,"Chicago-Naperville-Elgin, IL-IN-WI",16980.0,286.0,28600986.0,0.842,9.999655
4,Brain and Other Nervous System,"Cincinnati, OH-KY-IN",17140.0,18.0,4383144.0,0.842,4.106641


In [238]:
#creating seer and cdc compbined dataframe for scatter plot
CDC_SEER_df = Cancer_df.merge( seer_Cancer_df, how = 'left',left_on = 'Cancer_Sites', right_on = 'Cancer_Sites')
CDC_SEER_df = CDC_SEER_df.drop(['Race','Gender','index'], axis = 1)

CDC_SEER_df.head()

,Cancer_Sites,Count,5-Year_Conditional_Relative_Survival
0,Bones and Joints,907.0,0.912
1,Brain and Other Nervous System,10614.0,0.842
2,Colon and Rectum,120449.0,0.902
3,Endocrine System,33096.0,0.995
4,Esophagus,10199.0,0.757


In [240]:
#Sorting dataframes so data shows in highest to lowest format
Gender_Cancer_df = Gender_Cancer_df.sort_values(by = 'Count', ascending = False)
Age_Cancer_df = Age_Cancer_df.sort_values(by = 'Count', ascending = False)
Race_Cancer_df = Race_Cancer_df.sort_values(by = 'Count', ascending = False)

In [241]:
#Exporting dataframes to cvs files for use in HTML
seer_Gender_Cancer_df.to_csv('seer_Gender_Cancer.csv',index=False, encoding='utf-8-sig')
seer_Race_Cancer_df.to_csv('seer_Race_Cancer.csv',index=False, encoding='utf-8-sig')
MSA_complete_df.to_csv('MSA_complete.csv',index=False, encoding='utf-8-sig')
Race_Cancer_df.to_csv('Race_Cancer.csv',index=False, encoding='utf-8-sig')
Age_Cancer_df.to_csv('Age_Cancer.csv',index=False, encoding='utf-8-sig')
Gender_Cancer_df.to_csv('Gender_Cancer.csv',index=False, encoding='utf-8-sig')
MSA_Cancer_df.to_csv('MSA_Cancer.csv',index=False, encoding='utf-8-sig')
Cancer_df.to_csv('Cancer_summary.csv',index=False, encoding='utf-8-sig')
seer_total_df.to_csv('seer_total.csv',index=False, encoding='utf-8-sig')
CDC_SEER_df.to_csv('cancer_complete.csv',index=False, encoding='utf-8-sig')